# Transfer Learning to classify keep or throwaway necklace model with VGG16

After deciding to use the VGG16 model, my goal was to then decide what kind of labels I on the images I would use. Did I need to categorize the necklaces by type, as Etsy had done? Or could I simply just separate necklace images from non-necklace images?   
  
This model tests the latter: I train the model to identify whether there was a necklace in the image that would like a picture someone would upload, or not.   
  
Once I do this, I then use the model to produce feature vectors over which I calculate the similarity. At this point, I can compare the results to that produced by a model trained on categorical labels.  
  
*This code was written for use on Google Colab.*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import os
import time

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras import metrics, optimizers
%tensorflow_version 1.x
K.clear_session()

## Define paths & constants

In [ ]:
data_base_path = "/content/drive/My Drive/data/labeled/"
save_dir = os.path.abspath("/content/drive/My Drive/Colab Notebooks/")
print(os.listdir(data_base_path))

In [ ]:
# Ensure data directory paths are correct
print(len(os.listdir(data_base_path+"throwaway/")))
print(len(os.listdir(data_base_path+"keep/")))

In [ ]:
# Decide on constants for the model
nb_classes = 2
batch_size = 32
nb_train_samples = batch_size*3
nb_validation_samples = batch_size*1
nb_epochs = 100

img_height = 150
img_width = 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

## Build model

In [ ]:
# Extract base of VGG16 model:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=input_shape)

In [ ]:
# Add 3 trainable layers on top:
# (Tried adding different combination of additional layers &
# dropout layers... this configuration yielded the best metrics)
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(512*4, activation='relu'))
model.add(Dense(512*4, activation='relu'))
model.add(Dense(512*4, activation='relu'))

# predict category
model.add(Dense(nb_classes, activation='softmax'))

conv_base.trainable = True
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

## Image augmentation
  
Randomly generate training & validation data by performing horizontal flips, rotating, and stretching photos

In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255, 
    horizontal_flip=True, 
    rotation_range=40, 
    width_shift_range=.15,
    height_shift_range=.15,
    validation_split=0.2) # set 80/20 train/test data split

#test_datagen = ImageDataGenerator(rescale=1./255)

Produce train & validation data

In [ ]:
train_generator = train_gen.flow_from_directory(
    data_base_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data
    # shuffles by default

validation_generator = train_gen.flow_from_directory(
    data_base_path, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

### Inspect a batch

In [ ]:
sample_training_images, _ = next(train_generator)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

## Compile transfer learning model

In [ ]:
model.summary()

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


## Compile transfer learning model

### Hyperparameter Tuning
Tried a variety of learning rates, number of epochs, and both SGD & Adam optimizers. Please see ./tuning_binary_model/model_tuning.xlsx for a summary of changes in model accuracy with tuning.

In [ ]:
# Tried a variety of 
#LR1 = 0.001 # default with adam
#LR2 = 0.01
adam = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#sgd = optimizers.SGD(learning_rate=LR1, momentum=0.0, nesterov=False)
#model.compile(optimizer = sgd, 
model.compile(optimizer = adam, 
              loss = 'sparse_categorical_crossentropy',  # for labeled data
              metrics=['acc',f1_m,precision_m, recall_m])

## Train model

In [ ]:
start_time = time.time()
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=nb_epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
end_time = time.time()
print("Total Time: "+str(end_time - start_time)+" seconds.");''
os.system('say "your program has finished"')

In [1]:
# ensure path where model is saved is correct
#save_dir = os.path.join(os.getcwd(),'/drive/My Drive/Colab Notebooks/models/')
print(save_dir)

NameError: name 'save_dir' is not defined

In [ ]:
import pickle
model.save('/content/drive/My Drive/data/VGG_binaryclassifier_v1_updated_4layers_LR0.0001_13000_imgtraining.h5')
fit_history.history['time_total'] = end_time - start_time
pickle.dump(fit_history.history,open('/content/drive/My Drive/data/VGG_binaryclassifier_v1_updated_4layers_LR0.0001_13000_imgtraining.pkl','wb'))

## Visualize model metrics

Also visual model metrics in  
./tuning_binary_model/compare_categorical_binary_stats.ipynb & 
./tuning_binary_model/recover_stats_model.ipynb

In [ ]:
#print(fit_history.history.keys())
acc = fit_history.history['acc']
val_acc = fit_history.history['val_acc']

loss = fit_history.history['loss']
val_loss = fit_history.history['val_loss']

epochs_range = range(nb_epochs)

plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
#plt.savefig('./VGG.binary.v7.png')
plt.show()

In [ ]:
#'f1_m', 'precision_m', 'recall_m', 'val_loss', 'val_acc', 'val_f1_m', 'val_precision_m', 'val_recall_m'
#print(fit_history.history.keys())
acc = fit_history.history['precision_m']
val_acc = fit_history.history['val_precision_m']

loss = fit_history.history['recall_m']
val_loss = fit_history.history['val_recall_m']

epochs_range = range(nb_epochs)

plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Precision')
plt.plot(epochs_range, val_acc, label='Validation Precision')
plt.legend(loc='lower right')
plt.title('Precision')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Recall')
plt.plot(epochs_range, val_loss, label='Validation Recall')
plt.legend(loc='upper right')
plt.title('Recall')
#plt.savefig('./VGG.binary.v7.png')
plt.show()